# create camera groups

In [ ]:
# create camera groups
from sklearn.cluster import KMeans
import json
import numpy as np

def KMean(xyz, n_clusters):
    kmeans = KMeans(n_clusters = n_clusters, n_init=10)
    kmeans.fit(xyz)
    labels = kmeans.labels_
    
    clusters = []
    for i in range(n_clusters):
        idx = np.where(labels==i)[0]
        clusters.append(idx.astype(np.uint8))

    return clusters

root_dir = "datasets/meeting_room/MeetRoom/" #Your scene path
cur_frame = "colmap_0"
scene_name_list = ["discussion", "vrheadset"]
gs_mode= "3dgs_rade"
for scene_name in scene_name_list:
    cur_frame_dir = os.path.join(root_dir, scene_name, cur_frame)

    cameras_json_path = os.path.join(cur_frame_dir, gs_mode, "cameras.json")
    with open(cameras_json_path) as f:
        cameras_data = json.load(f)
    xyzs = []
    for cam in cameras_data:
        xyz = np.array(cam["position"])
        xyzs.append(xyz)
    xyzs = np.stack(xyzs, axis=0)
    print(xyzs.shape)

    groups = KMean(xyzs, 4)

    groups_list = [ i.tolist() for i in groups]
    print( scene_name, groups_list) 
    filename= os.path.join(root_dir, scene_name, "group.json")
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(groups_list, f, ensure_ascii=False)

# Generate Training Pair

In [ ]:
'''{
    "scene_name":"cur_roasted_beef_colmap",
    "cur_frame":"colmap_0",
    "next_frame""colmap_1",
}'''
import random
import os
import json

ROOT_DIR ="/workspace/my_folder/yjb/datasets/neural_3D/"
name_list= ["scene1",  "scene2"] #scene list

total_item_list = []
gap = 30 # the paper use 5 gap
for name in name_list:
    cur_list = []
    for start_idx in range(300):
        for end_idx in range(start_idx+1, min(300,start_idx+gap)):
            cur_list.append({"scene_name":name, "cur_frame": f"colmap_{start_idx}", "next_frame": f"colmap_{end_idx}"})
            cur_list.append({"scene_name":name, "cur_frame": f"colmap_{end_idx}", "next_frame": f"colmap_{start_idx}"})
    total_item_list.extend(cur_list)
print(len(total_item_list))
random.shuffle(total_item_list)
train_list = total_item_list[:-int(len(total_item_list)*0.01)]
val_list = total_item_list[-int(len(total_item_list)*0.01):]
print(len(val_list))
full_dict = {"train":train_list, "val": val_list}

filename = os.path.join(ROOT_DIR, "YOUR_TRAIN_NAME.json")
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(full_dict, f, ensure_ascii=False,indent=4)